### 1. MAG Quality Control


In [ ]:
#Estimate and assess the purity of our dataset with BUSCO. Same code as in W4.
#Bacteria
! qiime annotate fetch-busco-db \
    --p-lineages bacteria_odb12 \
    --o-db $data_dir/busco-db-bacteria.qza

#Fungi
qiime annotate fetch-busco-db \
  --p-lineages fungi_odb12 \
  --o-db $data_dir/busco-db-fungi.qza

#Archaea
qiime annotate fetch-busco-db \
    --p-lineages archaea_odb12 \
    --o-db $data_dir/busco-db-archaea.qza


#Next, we use the database we fetched to run BUSCO with our recovered MAGs as input:
! qiime annotate evaluate-busco \
    --i-mags $data_dir/updog_mags.qza \
    --i-db $data_dir/busco-db-bacteria.qza \
    --p-lineage-dataset bacteria_odb12 \
    --p-cpu 3 \
    --o-results $data_dir/busco-results-bacteria.qza \
    --o-visualization $data_dir/mags-busco-bacteria.qzv

#This should generate a table
#q2.Visualization.load(f"{data_dir}/mags.qzv")

#Now that we evaluated the quality of our MAGs, we can use this information to filter out only the best ones. Maybe we have to adjust some parameters...
# --- Bacteria ---
mosh annotate filter-mags \
  --i-mags $data_dir/mags.qza \
  --m-metadata-file $data_dir/busco-results-bacteria.qza \
  --p-where 'complete>50' \
  --p-no-exclude-ids \
  --p-on mag \
  --o-filtered-mags $data_dir/mags_filtered_bacteria_50.qza \
  --verbose

# --- Archaea ---
mosh annotate filter-mags \
  --i-mags $data_dir/mags.qza \
  --m-metadata-file $data_dir/busco-results-archaea.qza \
  --p-where 'complete>50' \
  --p-no-exclude-ids \
  --p-on mag \
  --o-filtered-mags $data_dir/mags_filtered_archaea_50.qza \
  --verbose

# --- Fungi ---
mosh annotate filter-mags \
  --i-mags $data_dir/mags.qza \
  --m-metadata-file $data_dir/busco-results-fungi.qza \
  --p-where 'complete>50' \
  --p-no-exclude-ids \
  --p-on mag \
  --o-filtered-mags $data_dir/mags_filtered_fungi_50.qza \
  --verbose